# Foundations of Computer Science Project

Virginia Gallù
Marco Guarisco
Camilla Marchetti


### Libraries used

In [1]:
import pandas as pd
import numpy as np
import datetime
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Read data

In [2]:
dogs=pd.read_csv("dogs.csv")
travels=pd.read_csv("dogTravel.csv")
states=pd.read_csv("NST-EST2021-POP.csv",names=['state', 'population'])

In [3]:
dogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,Las Vegas,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,Mesquite,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Pahrump,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,Henderson,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09


In [4]:
travels.head()

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Abacos,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Adam,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN


In [5]:
states.head()

,state,population
0,Alabama,5.024.279
1,Alaska,733.391
2,Arizona,7.151.502
3,Arkansas,3.011.524
4,California,39.538.223


Here we remove the points from the column "population" 

In [6]:
states['population']=states['population'].str.replace('.', '')
states

C:\Users\marcg\AppData\Local\Temp\ipykernel_10344\261990245.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  states['population']=states['population'].str.replace('.', '')


,state,population
0,Alabama,5024279
1,Alaska,733391
2,Arizona,7151502
3,Arkansas,3011524
4,California,39538223
5,Colorado,5773714
6,Connecticut,3605944
7,Delaware,989948
8,District of Columbia,689545
9,Florida,21538187


## 1. Extract all dogs with status that is not adoptable

In [7]:
dogs[dogs['status']!='adoptable']

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,NV,89146,US,89009,2019-09-20,NaN,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,AZ,85249,US,AZ,2019-09-20,NaN,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,NY,12220,US,CT,2019-09-20,NaN,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,NY,12220,US,CT,2019-09-20,NaN,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,NY,12220,US,CT,2019-09-20,NaN,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
12600,45229004,NY1436,https://www.petfinder.com/dog/elmo-momo-452290...,Dog,Dog,American Bulldog,NaN,True,False,NaN,...,NY,12477,US,CT,2019-09-20,NaN,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
12613,45227052,NY1436,https://www.petfinder.com/dog/bianca-pinky-452...,Dog,Dog,Mixed Breed,NaN,False,False,White / Cream,...,NY,12477,US,CT,2019-09-20,NaN,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
17619,45569380,CA1209,https://www.petfinder.com/dog/baby-girl-455693...,Dog,Dog,Maltese,NaN,False,False,White / Cream,...,VA,20136,US,DC,2019-09-20,NaN,Dog,This 10-year young senior is very sweet and lo...,76,263.63
18611,44694387,MD295,https://www.petfinder.com/dog/king-bert-bertie...,Dog,Dog,Fox Terrier,Chihuahua,True,False,Bicolor,...,MD,20905,US,DC,2019-09-20,NaN,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
19747,36978896,VA127,https://www.petfinder.com/dog/maddie-cutie-pat...,Dog,Dog,Alaskan Malamute,NaN,False,False,Bicolor,...,PA,17325,US,DC,2019-09-20,NaN,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


In the dogs dataframe we find that there is an error when the value in the column status is not "adoptable". In those rows every value is shifted one column to the left. Here we fix this.

In [8]:
baddogs=dogs[dogs['status']!='adoptable']
baddogs.insert(25,'status0', 'not adoptable')
betterdogs=baddogs.rename(columns={'status':'posted0','posted':'contact_city0', 'contact_city':'contact_state0', 'contact_state':'contact_zip0', 'contact_zip':'contact_country0', 'contact_country':'stateQ0', 'stateQ':'accessed0', 'accessed': 'eliminare'})
evenbetterdogs=betterdogs.rename(columns={'status0':'status', 'posted0':'posted','contact_city0':'contact_city','contact_state0':'contact_state','contact_zip0':'contact_zip','contact_country0':'contact_country','stateQ0':'stateQ','accessed0':'accessed'})
gooddogs=evenbetterdogs.drop(columns=['eliminare'])
gooddogs.head()

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61


In [9]:
cani=dogs
cani.loc[cani['status']!='adoptable']=gooddogs
cani[cani['status']!='adoptable']

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_city,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost
644,41330726,NV173,https://www.petfinder.com/dog/gunther-gunny-41...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,Las Vegas,NV,89146,US,89009,2019-09-20,Dog,Meet handsome 3 year old Gunther. Gunther came...,108,256.88
5549,38169117,AZ414,https://www.petfinder.com/dog/annabelle-annie-...,Dog,Dog,Boxer,Pit Bull Terrier,True,False,Black,...,Chandler,AZ,85249,US,AZ,2019-09-20,Dog,You can fill out an adoption application onlin...,80,130.77
10888,45833989,NY98,https://www.petfinder.com/dog/pepper-courtesy-...,Dog,Dog,Beagle,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,This is Pepper. He is a 15 year old tri-color ...,86,180.70
11983,45515547,NY98,https://www.petfinder.com/dog/cooper-courtesy-...,Dog,Dog,Mixed Breed,NaN,False,False,NaN,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"Cooper is 13 years old, but according to a ver...",105,400.82
12495,45294115,NY98,https://www.petfinder.com/dog/daisy-courtesy-l...,Dog,Dog,Basset Hound,NaN,False,False,Brown / Chocolate,...,Albany,NY,12220,US,CT,2019-09-20,Dog,"â¢Basset Hound, female, â¢10 years \n\nDelig...",57,82.61
12600,45229004,NY1436,https://www.petfinder.com/dog/elmo-momo-452290...,Dog,Dog,American Bulldog,NaN,True,False,NaN,...,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello i'm MoMo or Elmo , 7 year old, mixed bre...",73,136.30
12613,45227052,NY1436,https://www.petfinder.com/dog/bianca-pinky-452...,Dog,Dog,Mixed Breed,NaN,False,False,White / Cream,...,Saugerties,NY,12477,US,CT,2019-09-20,Dog,"Hello I'm Bianca, a female, 7 year old mixed b...",107,231.31
17619,45569380,CA1209,https://www.petfinder.com/dog/baby-girl-455693...,Dog,Dog,Maltese,NaN,False,False,White / Cream,...,Bristow,VA,20136,US,DC,2019-09-20,Dog,This 10-year young senior is very sweet and lo...,76,263.63
18611,44694387,MD295,https://www.petfinder.com/dog/king-bert-bertie...,Dog,Dog,Fox Terrier,Chihuahua,True,False,Bicolor,...,Silver Spring,MD,20905,US,DC,2019-09-20,Dog,"\""Bertie\"" came to us from the shelter. He wa...",61,158.84
19747,36978896,VA127,https://www.petfinder.com/dog/maddie-cutie-pat...,Dog,Dog,Alaskan Malamute,NaN,False,False,Bicolor,...,Gettysburg,PA,17325,US,DC,2019-09-20,Dog,Maddie is our little Miss Cutie Patootie! She ...,119,431.66


We created a new dataframe named cani, in which the forementioned problem is fixed.

## 2. For each (primary) breed, determine the number of dogs

In [10]:
dogs.groupby('breed_primary').count()['id']

breed_primary
Affenpinscher                         17
Afghan Hound                           4
Airedale Terrier                      19
Akbash                                 3
Akita                                181
                                    ... 
Wirehaired Pointing Griffon            1
Wirehaired Terrier                    60
Xoloitzcuintli / Mexican Hairless     11
Yellow Labrador Retriever            158
Yorkshire Terrier                    360
Name: id, Length: 216, dtype: int64

## 3. For each (primary) breed, determine the ratio between the number of dogs of Mixed Breed and those not of Mixed Breed. Hint: look at the secondary_breed.

In [11]:
mixed=dogs[dogs['breed_secondary'].notna()].groupby('breed_primary').count()
not_mixed=dogs[dogs['breed_secondary'].isnull()].groupby('breed_primary').count()

mixed['id']/not_mixed['id']

breed_primary
Affenpinscher                        0.133333
Afghan Hound                         0.333333
Airedale Terrier                     0.900000
Akbash                                    NaN
Akita                                0.403101
                                       ...   
Wirehaired Pointing Griffon               NaN
Wirehaired Terrier                   0.428571
Xoloitzcuintli / Mexican Hairless    0.571429
Yellow Labrador Retriever            0.645833
Yorkshire Terrier                    0.558442
Name: id, Length: 216, dtype: float64

In [12]:
mixed_breed=dogs[dogs['breed_mixed']==True].groupby('breed_primary').count()
not_mixed_breed=dogs[dogs['breed_mixed']==False].groupby('breed_primary').count()

mixed_breed['id']/not_mixed_breed['id']

breed_primary
Affenpinscher                        0.416667
Afghan Hound                              NaN
Airedale Terrier                     8.500000
Akbash                               2.000000
Akita                                0.846939
                                       ...   
Wirehaired Pointing Griffon               NaN
Wirehaired Terrier                   3.000000
Xoloitzcuintli / Mexican Hairless    0.833333
Yellow Labrador Retriever            3.388889
Yorkshire Terrier                    1.292994
Name: id, Length: 216, dtype: float64

We get different results because there in not a match between having a true mixed_breed value and a not null value in the column breed_secondary. 

## 4. For each (primary) breed, determine the earliest and the latest posted timestamp.

In [13]:
pd.to_datetime(cani['posted'],errors= 'coerce' ,format='%Y-%m-%dT%H:%M:%S%z')

0       2019-09-20 16:37:59+00:00
1       2019-09-20 16:24:57+00:00
2       2019-09-20 14:10:11+00:00
3       2019-09-20 10:08:22+00:00
4       2019-09-20 06:48:30+00:00
                   ...           
58175   2019-05-03 14:23:49+00:00
58176   2019-04-13 16:20:24+00:00
58177   2018-09-27 04:18:56+00:00
58178   2018-09-12 05:03:38+00:00
58179   2018-09-03 20:42:24+00:00
Name: posted, Length: 58180, dtype: datetime64[ns, UTC]

In [14]:
sorted = cani.sort_values('posted')

In [15]:
cani.groupby('breed_primary')['posted'].min()

breed_primary
Affenpinscher                        2012-03-08T10:27:33+0000
Afghan Hound                         2017-06-29T23:28:51+0000
Airedale Terrier                     2014-06-13T12:59:36+0000
Akbash                               2019-07-21T00:35:59+0000
Akita                                2012-03-03T09:31:08+0000
                                               ...           
Wirehaired Pointing Griffon          2016-06-29T20:03:55+0000
Wirehaired Terrier                   2012-11-27T14:07:54+0000
Xoloitzcuintli / Mexican Hairless    2007-02-01T00:00:00+0000
Yellow Labrador Retriever            2010-05-31T00:00:00+0000
Yorkshire Terrier                    2012-02-04T17:14:52+0000
Name: posted, Length: 216, dtype: object

In [16]:
cani.groupby('breed_primary')['posted'].max()

breed_primary
Affenpinscher                        2019-09-14T10:10:51+0000
Afghan Hound                         2019-07-27T00:38:48+0000
Airedale Terrier                     2019-09-19T18:40:39+0000
Akbash                               2019-08-23T17:11:04+0000
Akita                                2019-09-20T15:19:57+0000
                                               ...           
Wirehaired Pointing Griffon          2016-06-29T20:03:55+0000
Wirehaired Terrier                   2019-09-19T22:52:45+0000
Xoloitzcuintli / Mexican Hairless    2019-09-08T11:15:54+0000
Yellow Labrador Retriever            2019-09-20T06:30:27+0000
Yorkshire Terrier                    2019-09-20T16:56:38+0000
Name: posted, Length: 216, dtype: object

## 5. For each state, compute the sex imbalance, that is the difference between male and female dogs. In which state this imbalance is largest?

In [17]:
male=cani[cani['sex']=='Male'].groupby('contact_state').size()
female=cani[cani['sex']=='Female'].groupby('contact_state').size()
diff=male-female
diff

contact_state
AK      1
AL     -4
AR     -7
AZ    114
CA    110
CO    -51
CT     58
DC    -16
DE      0
FL    101
GA     25
HI     13
IA     81
ID      1
IL    101
IN    178
KS      0
KY    109
LA     23
MA     28
MD    136
ME    -29
MI     39
MN     34
MO     70
MS    -40
MT      4
NB      0
NC     63
ND      8
NE     -2
NH     -9
NJ     56
NM     17
NV     54
NY    117
OH    205
OK      8
OR     25
PA     83
QC      0
RI     -5
SC    100
SD     10
TN    119
TX     -8
UT     79
VA    159
VT     42
WA     88
WI     12
WV    101
WY     10
dtype: int64

In [18]:
diff.idxmax(),diff[diff.idxmax()]

('OH', 205)

The maximum imbalance is found in the state of Ohio

## 6. For each pair (age, size), determine the average duration of the stay and the average cost of stay.

In [19]:
cani.groupby(['age','size']).mean()['stay_duration']

age     size       
Adult   Extra Large    89.015414
        Large          89.531943
        Medium         89.421036
        Small          89.407479
Baby    Extra Large    87.032967
        Large          89.701564
        Medium         89.577668
        Small          89.958291
Senior  Extra Large    88.861111
        Large          88.984298
        Medium         89.810052
        Small          89.073626
Young   Extra Large    90.586345
        Large          90.104206
        Medium         89.515123
        Small          89.814275
Name: stay_duration, dtype: float64

In [20]:
cani.groupby(['age','size']).mean()['stay_cost']

age     size       
Adult   Extra Large    232.591561
        Large          238.661141
        Medium         238.258977
        Small          238.974838
Baby    Extra Large    237.180879
        Large          238.698827
        Medium         237.108131
        Small          239.083810
Senior  Extra Large    235.232361
        Large          237.507364
        Medium         238.514615
        Small          238.282286
Young   Extra Large    245.835582
        Large          238.149506
        Medium         239.304603
        Small          241.540069
Name: stay_cost, dtype: float64

## 7. Find the dogs involved in at least 3 travels. Also list the breed of those dogs.

In [21]:
cani_breed=cani[['id','breed_primary']]
viaggi=pd.merge(cani_breed, travels, on='id')
viaggi_cani=viaggi.groupby(['id','breed_primary'], as_index=False).size()
viaggi_cani[viaggi_cani['size']>2]


,id,breed_primary,size
5,16657005,Pit Bull Terrier,4
9,20905974,Chow Chow,5
17,24894870,Hound,4
18,24894894,Hound,4
55,33218331,Alaskan Malamute,7
...,...,...,...
4110,46042569,Labrador Retriever,3
4111,46042587,Labrador Retriever,3
4112,46042618,Labrador Retriever,3
4113,46043099,Labrador Retriever,3


## 8. Fix the travels table so that the correct state is computed from the manual and the found fields. If manual is not missing, then it overrides what is stored in found.

With loc method, we can access a group of rows or columns with a condition or a boolean array. If we can access it we can also manipulate the values.

In [22]:
travels.loc[travels['manual'].notna(), 'found'] = travels['manual']

In [23]:
travels

,index,id,contact_city,contact_state,description,found,manual,remove,still_there
0,0,44520267,Anoka,MN,Boris is a handsome mini schnauzer who made hi...,Arkansas,NaN,NaN,NaN
1,1,44698509,Groveland,FL,Duke is an almost 2 year old Potcake from Abac...,Bahamas,Bahamas,NaN,NaN
2,2,45983838,Adamstown,MD,Zac Woof-ron is a heartthrob movie star lookin...,Maryland,Maryland,NaN,NaN
3,3,44475904,Saint Cloud,MN,~~Came in to the shelter as a transfer from an...,Adaptil,NaN,True,NaN
4,4,43877389,Pueblo,CO,Palang is such a sweetheart. She loves her peo...,Afghanistan,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
6189,6189,40492179,Fairmont,WV,Please contact Pet (information@pethelpersinc....,WV,NaN,True,NaN
6190,6190,45799729,Eagle Mountain,UT,Shiny is an approximately 4-6-year-old spayed ...,Wyoming,NaN,NaN,NaN
6191,6191,34276515,Newnan,GA,Yanni is a Male Great Pyrenees that we rescue...,Yazmin,NaN,True,NaN
6192,6192,44519341,Dayton,OH,Callie is a 14 year old Chihuahua whose owner ...,Ohio,Ohio,NaN,NaN


## 9. For each state, compute the ratio between the number of travels and the population.

To compute the ratio between the number of travels and the population of each state we decide to use the column contact_state from the travels dataframe. In order to merge travels and states we need to create a new column in the states dataframe that contains the abbraviated states names.

We define first a function that creates a new column with the abbreviated names

In [24]:
def add_state_abbreviations(df, state_col):
    state_abbreviations = {
        'Alabama': 'AL',
        'Alaska': 'AK',
        'Arizona': 'AZ',
        'Arkansas': 'AR',
        'California': 'CA',
        'Colorado': 'CO',
        'Connecticut': 'CT',
        'Delaware': 'DE',
        'Florida': 'FL',
        'Georgia': 'GA',
        'Hawaii': 'HI',
        'Idaho': 'ID',
        'Illinois': 'IL',
        'Indiana': 'IN',
        'Iowa': 'IA',
        'Kansas': 'KS',
        'Kentucky': 'KY',
        'Louisiana': 'LA',
        'Maine': 'ME',
        'Maryland': 'MD',
        'Massachusetts': 'MA',
        'Michigan': 'MI',
        'Minnesota': 'MN',
        'Mississippi': 'MS',
        'Missouri': 'MO',
        'Montana': 'MT',
        'Nebraska': 'NE',
        'Nevada': 'NV',
        'New Hampshire': 'NH',
        'New Jersey': 'NJ',
        'New Mexico': 'NM',
        'New York': 'NY',
        'North Carolina': 'NC',
        'North Dakota': 'ND',
        'Ohio': 'OH',
        'Oklahoma': 'OK',
        'Oregon': 'OR',
        'Pennsylvania': 'PA',
        'Rhode Island': 'RI',
        'South Carolina': 'SC',
        'South Dakota': 'SD',
        'Tennessee': 'TN',
        'Texas': 'TX',
        'Utah': 'UT',
        'Vermont': 'VT',
        'Virginia': 'VA',
        'Washington': 'WA',
        'West Virginia': 'WV',
        'Wisconsin': 'WI',
        'Wyoming': 'WY',
    }
    df['state_abbr'] = df[state_col].map(state_abbreviations)


We can apply the function to our states dataframe.

In [25]:
add_state_abbreviations(states, "state")
states.head()

,state,population,state_abbr
0,Alabama,5024279,AL
1,Alaska,733391,AK
2,Arizona,7151502,AZ
3,Arkansas,3011524,AR
4,California,39538223,CA


We can now merge travels and states and compute the ratio between number of travels and state population.

In [26]:
mondo=pd.merge(travels, states, left_on='contact_state', right_on='state_abbr')
mondo

viaggixstato=mondo.groupby(['state','population'],as_index=False).size()
viaggixstato['ratio']=viaggixstato['size']/viaggixstato['population'].astype(int)
viaggixstato

,state,population,size,ratio
0,Alabama,5024279,75,1.492751e-05
1,Arizona,7151502,70,9.788154e-06
2,Arkansas,3011524,10,3.320578e-06
3,California,39538223,28,7.081755e-07
4,Colorado,5773714,103,1.783947e-05
5,Connecticut,3605944,90,2.495879e-05
6,Delaware,989948,57,5.757878e-05
7,Florida,21538187,133,6.175079e-06
8,Georgia,10711908,109,1.017559e-05
9,Illinois,12812508,37,2.887803e-06


## 10. For each dog, compute the number of days from the posted day to the day of last access.

We first parse the columns to the right datetime format.

In [27]:
pd.to_datetime(cani['accessed'], format='%Y-%m-%d')

0       2019-09-20
1       2019-09-20
2       2019-09-20
3       2019-09-20
4       2019-09-20
           ...    
58175   2019-09-20
58176   2019-09-20
58177   2019-09-20
58178   2019-09-20
58179   2019-09-20
Name: accessed, Length: 58180, dtype: datetime64[ns]

In [28]:
cani['posted_date']=pd.to_datetime(cani['posted'],errors= 'coerce' ,format='%Y-%m-%dT%H:%M:%S%z').dt.strftime('%Y-%m-%d')
cani

,id,org_id,url,type.x,species,breed_primary,breed_secondary,breed_mixed,breed_unknown,color_primary,...,contact_state,contact_zip,contact_country,stateQ,accessed,type.y,description,stay_duration,stay_cost,posted_date
0,46042150,NV163,https://www.petfinder.com/dog/harley-46042150/...,Dog,Dog,American Staffordshire Terrier,Mixed Breed,True,False,White / Cream,...,NV,89147,US,89009,2019-09-20,Dog,Harley is not sure how he wound up at shelter ...,70,124.81,2019-09-20
1,46042002,NV163,https://www.petfinder.com/dog/biggie-46042002/...,Dog,Dog,Pit Bull Terrier,Mixed Breed,True,False,Brown / Chocolate,...,NV,89147,US,89009,2019-09-20,Dog,6 year old Biggie has lost his home and really...,49,122.07,2019-09-20
2,46040898,NV99,https://www.petfinder.com/dog/ziggy-46040898/n...,Dog,Dog,Shepherd,NaN,False,False,Brindle,...,NV,89027,US,89009,2019-09-20,Dog,Approx 2 years old.\n Did I catch your eye? I ...,87,281.51,2019-09-20
3,46039877,NV202,https://www.petfinder.com/dog/gypsy-46039877/n...,Dog,Dog,German Shepherd Dog,NaN,False,False,NaN,...,NV,89048,US,89009,2019-09-20,Dog,NaN,62,145.83,2019-09-20
4,46039306,NV184,https://www.petfinder.com/dog/theo-46039306/nv...,Dog,Dog,Dachshund,NaN,False,False,NaN,...,NV,89052,US,89009,2019-09-20,Dog,Theo is a friendly dachshund mix who gets alon...,93,241.09,2019-09-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58175,44605893,WY20,https://www.petfinder.com/dog/tren-44605893/wy...,Dog,Dog,Border Collie,NaN,False,False,"Tricolor (Brown, Black, & White)",...,WY,82520,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,324.34,2019-05-03
58176,44457061,WY24,https://www.petfinder.com/dog/harley-44457061/...,Dog,Dog,Australian Shepherd,Australian Cattle Dog / Blue Heeler,True,False,NaN,...,WY,82501,US,WY,2019-09-20,Dog,NaN,65,245.90,2019-04-13
58177,42865848,WY20,https://www.petfinder.com/dog/echo-42865848/wy...,Dog,Dog,Border Collie,NaN,False,False,NaN,...,WY,82637,US,WY,2019-09-20,Dog,"Due to the small size of our volunteer base, w...",100,184.06,2018-09-27
58178,42734734,WY24,https://www.petfinder.com/dog/simon-42734734/w...,Dog,Dog,Boxer,Mixed Breed,True,False,NaN,...,WY,82501,US,WY,2019-09-20,Dog,NaN,58,61.05,2018-09-12


Then we can compute the number of days from the posted day to the day of last access.

In [29]:
cani[['accessed','posted_date']] = cani[['accessed','posted_date']].apply(pd.to_datetime)

cani['diff']=(cani['accessed']-cani['posted_date']).dt.days
cani[['id','diff']]

,id,diff
0,46042150,0
1,46042002,0
2,46040898,0
3,46039877,0
4,46039306,0
...,...,...
58175,44605893,140
58176,44457061,160
58177,42865848,358
58178,42734734,373


## 11. Partition the dogs according to the number of weeks from the posted day to the day of last access.

We simply divide the number of days by 7 to compute the number of weeks.

In [30]:
cani['weeks']=cani['diff']//7
settimane=cani.sort_values(by=['weeks'])[['weeks','id']]
settimane=settimane.set_index('weeks')
settimane

,id
weeks,
0,46042150
0,45968226
0,45968176
0,45968114
0,45968101
...,...
729,5142790
746,4527948
811,2613506


## 12. Find for duplicates in the dogs dataset. Two records are duplicates if they have (1) same breeds and sex, and (2) they share at least 90% of the words in the description field. Extra points if you find and implement a more refined for determining if two rows are duplicates.

We first eliminate all the rows that don't have a description.

In [31]:
cani_descriptions=cani[cani['description'].notna()]

We use the cosine similarity method

In [32]:
def find_similar_strings(df, column):
  # Convert the strings in the specified column to vectors using a CountVectorizer
  vectorizer = CountVectorizer()
  vectors = vectorizer.fit_transform(df[column])
  
  # Calculate the cosine similarity between all pairs of vectors
  similarity_matrix = cosine_similarity(vectors)
  
  # Initialize an empty list to store the similar strings
  similar_strings = []
  
  # Iterate over the rows and columns of the similarity matrix
  for i in range(similarity_matrix.shape[0]):
    for j in range(i+1, similarity_matrix.shape[1]):
      # If the similarity is greater than 0.9 and the strings are not the same, add them to the list followed by their index
      if similarity_matrix[i][j] > 0.9 and i != j:
        similar_strings.append((df.iloc[i][column], df.iloc[j][column],i,j))
        
  return similar_strings


In [33]:
coppie=cani_descriptions.groupby(['breed_primary','sex']).apply(find_similar_strings, column='description')

In [34]:
coppie

breed_primary                      sex   
Affenpinscher                      Female    [(Elsie.jpgThank you for looking at our availa...
                                   Male                                                     []
Afghan Hound                       Female                                                   []
                                   Male                                                     []
Airedale Terrier                   Female                                                   []
                                                                   ...                        
Xoloitzcuintli / Mexican Hairless  Male                                                     []
Yellow Labrador Retriever          Female    [(**WE ARE ALWAYS IN NEED OF FOSTER HOMES**  H...
                                   Male      [(Duke is a 4-year-old neutered male fox red l...
Yorkshire Terrier                  Female    [(Lady is such a sweet girl. She loves to be i...
        

In [35]:
coppiedf=pd.DataFrame(coppie)

In [36]:
coppiedf

0
breed_primary                     sex                                                      
Affenpinscher                     Female  [(Elsie.jpgThank you for looking at our availa...
                                  Male                                                   []
Afghan Hound                      Female                                                 []
                                  Male                                                   []
Airedale Terrier                  Female                                                 []
...                                                                                     ...
Xoloitzcuintli / Mexican Hairless Male                                                   []
Yellow Labrador Retriever         Female  [(**WE ARE ALWAYS IN NEED OF FOSTER HOMES**  H...
                                  Male    [(Duke is a 4-year-old neutered male fox red l...
Yorkshire Terrier                 Female  [(Lady is such a sweet girl. She loves to be i...
                                  Male    [(Kyle is a adorable little cuddle bug. He is ...

[403 rows x 1 columns]

In [37]:
coppiedf[0]

breed_primary                      sex   
Affenpinscher                      Female    [(Elsie.jpgThank you for looking at our availa...
                                   Male                                                     []
Afghan Hound                       Female                                                   []
                                   Male                                                     []
Airedale Terrier                   Female                                                   []
                                                                   ...                        
Xoloitzcuintli / Mexican Hairless  Male                                                     []
Yellow Labrador Retriever          Female    [(**WE ARE ALWAYS IN NEED OF FOSTER HOMES**  H...
                                   Male      [(Duke is a 4-year-old neutered male fox red l...
Yorkshire Terrier                  Female    [(Lady is such a sweet girl. She loves to be i...
        

In [38]:
sum_lengths = coppiedf[0].apply(lambda x: len(x)).sum()

print(sum_lengths) 

13603


In [48]:
coppiedf.iloc[403][0]

[('**WE ARE ALWAYS IN NEED OF FOSTER HOMES**  Homeward Bound Dog Rescue is an all volunteer rescue organization and all our dogs and puppies are in foster homes - we are not a shelter and do NOT have a facility where we house our dogs. For adoption information go to: www.homewardbounddogrescue.com.  You must fill out an application online as we do NOT accept applications at our clinics.\n\nHi, I am Kousa, a female Yellow Lab mix pup who is looking for a forever home.  I am a very happy, sweet girl so come and see me soon. \n\nOur adoption clinics are at our new facility in Schenectady on John Street, directly behind Boulevard Bowl (Erie Boulevard) every Saturday from 10 am to 1 pm (directions are on our website). We require an approved application before we consider anyone for an adoption - so filling out the application is your first step. We do NOT accept applications at our clinics, they MUST be filled out online before coming to our adoption events.  Different dogs and puppies go u